In [2]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import length
data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [4]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopwordsremover = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
cv = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tfidf')
class_num = StringIndexer(inputCol='class', outputCol='label')

clean_up = VectorAssembler(inputCols=['tfidf', 'length'], outputCol='features')

In [13]:
from pyspark.ml import Pipeline
data_pipeline = Pipeline(stages = [tokenizer, stopwordsremover, cv, idf, class_num, clean_up])
cleaned_data = data_pipeline.fit(data).transform(data).select('label', 'features')
cleaned_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [14]:
train_data, test_data = cleaned_data.randomSplit([0.7, 0.3])
from pyspark.ml.classification import NaiveBayes
model = NaiveBayes().fit(train_data)
results = model.transform(test_data)
results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,8,9...|[-799.84714978817...|[1.0,3.2452063348...|       0.0|
|  0.0|(13459,[0,1,2,14,...|[-611.54973074610...|[1.0,2.8590504487...|       0.0|
|  0.0|(13459,[0,1,2,44,...|[-1075.8624676379...|[1.0,1.4126781798...|       0.0|
|  0.0|(13459,[0,1,4,53,...|[-803.23992927190...|[1.0,6.8138429225...|       0.0|
|  0.0|(13459,[0,1,6,16,...|[-998.13107558111...|[1.0,6.5983076185...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-1170.9432459918...|[1.0,2.8519068602...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-568.35270741540...|[1.0,7.3140955928...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-568.35270741540...|[1.0,7.3140955928...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-882.27211441693...|[1.0,2.5855921035...|       0.0|
|  0.0|(13459,[0

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
print('Test accuracy:')
print(MulticlassClassificationEvaluator(metricName='accuracy').evaluate(results))
print('Train accuracy:')
print(MulticlassClassificationEvaluator(metricName='accuracy').evaluate(model.transform(train_data)))

Test accuracy:
0.9186046511627907
Train accuracy:
0.9959390862944163
